In [63]:
import re, sys, math, json, os, urllib.request
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 

#Loading the standalone dataset
try:
  model_data = pd.read_csv('inżynierka_standalone_data.csv')
except Exception as e:
    print('Data load error: ',e)

In [64]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import keras
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [65]:
raw_data = model_data
raw_data.head()

,Unnamed: 0,category,headline,labels
0,0,POLITICS,Trump's Crackdown On Immigrant Parents Puts Mo...,"([[1, 0, 1]], array([[0.03103773, 0.10634882, ..."
1,1,POLITICS,Trump's Son Should Be Concerned': FBI Obtained...,"([[0, 1, 1]], array([[0.04499885, 0.10186471, ..."
2,2,POLITICS,Edward Snowden: There's No One Trump Loves Mor...,"([[0, 1, 1]], array([[0.03465121, 0.1110402 , ..."
3,3,POLITICS,Booyah: Obama Photographer Hilariously Trolls ...,"([[0, 1, 1]], array([[0.01805554, 0.10873604, ..."
4,4,POLITICS,Ireland Votes To Repeal Abortion Amendment In ...,"([[0, 1, 1]], array([[0.02022719, 0.15641323, ..."


In [66]:
#Deleting an unnecessary column
raw_data = raw_data.drop('Unnamed: 0', axis=1)

In [67]:
#Deleting an unnecessary column - predictions shouldn't depend on pronounced categories
raw_data = raw_data.drop('category', axis=1)

In [68]:
#Getting only the predictions from labels
raw_data['labels'] = [eval('['+re.findall('\[\[(.*?)\]\]',row)[0]+']') for row in raw_data['labels']]

In [69]:
#Binary encodinf labels
raw_data['labels'] = [row[:-1] for row in raw_data['labels']]
raw_data['labels'] = [1 if row[0]==1 else 0 for row in raw_data['labels']]
raw_data

,headline,labels
0,Trump's Crackdown On Immigrant Parents Puts Mo...,1
1,Trump's Son Should Be Concerned': FBI Obtained...,0
2,Edward Snowden: There's No One Trump Loves Mor...,0
3,Booyah: Obama Photographer Hilariously Trolls ...,0
4,Ireland Votes To Repeal Abortion Amendment In ...,0
...,...,...
4995,Why We Are A Foster Parent Family,0
4996,This Is The Face Of A Mental Health Patient,0
4997,How Smart People Work Less And Get More Done,0
4998,15 Signs That You Are Emotionally Intelligent,0


In [70]:
import seaborn as sns

import re
from sklearn.model_selection import train_test_split
from string import punctuation
from collections import Counter
import numpy as np

#Getting only the title
titles = [re.split("^(.+?),", title)[-1] for title in raw_data.headline]
#Removing whitespace
titles = [re.sub(r'\s+', ' ', title) for title in titles]
encoded_labels = raw_data['labels']

#Dividing data into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(titles, encoded_labels, test_size=0.20, random_state=42)

titles

["Trump's Crackdown On Immigrant Parents Puts More Kids In An Already Strained System",
 "Trump's Son Should Be Concerned': FBI Obtained Wiretaps Of Putin Ally Who Met With Trump Jr.",
 "Edward Snowden: There's No One Trump Loves More Than Vladimir Putin",
 "Booyah: Obama Photographer Hilariously Trolls Trump's 'Spy' Claim",
 'Ireland Votes To Repeal Abortion Amendment In Landslide Referendum',
 "Ryan Zinke Looks To Reel Back Some Critics With 'Grand Pivot' To Conservation",
 "Trump's Scottish Golf Resort Pays Women Significantly Less Than Men: Report",
 ' But Taboo Sex Is Still Being Criminalized',
 "Bishop Michael Curry Joins Christian March To White House To 'Reclaim Jesus'",
 'How The Chinese Exclusion Act Can Help Us Understand Immigration Politics Today',
 'Roger Stone Pressed For Damaging Emails About Hillary Clinton From WikiLeaks: Report',
 'Publix Suspends Contributions To NRA-Backed Politician Amid Protests',
 ' Research Paper Says',
 'A Member Of The Far-Right Proud Boys Me

In [71]:
#Preparing testing and training dataframes
train_df = pd.DataFrame({
    'text': X_train,
    'labels':y_train
})

print(train_df)

eval_df = pd.DataFrame({
    'text': X_test,
    'labels': y_test
})

print([eval_df['labels']])

                                                   text  labels
4227  Steps You Can Take To Reduce Your Ovarian Canc...       0
4676  What I've Learned From Counseling Couples Thro...       0
800              And The Military Can't Enact Trump Ban       0
3671  Be Like Mike? 5 Things to Consider When You De...       0
4193  Constantly Checking Social Media During A Trag...       0
...                                                 ...     ...
4426  Aetna May Have Exposed The HIV Status Of Thous...       1
466   Sarah Huckabee Sanders Responds To Michelle Wo...       0
3092                                    Researchers Say       0
3772  Kids and Money: 4 Grocery Tips to Help Teach Y...       0
860   Vermont Governor Signs Gun Control Bills Into Law       0

[4000 rows x 2 columns]
[1501    0
2586    0
2653    0
1055    0
705     0
       ..
4711    0
2313    0
3214    0
2732    0
1926    0
Name: labels, Length: 1000, dtype: int64]


In [74]:
from simpletransformers.classification import ClassificationModel

#Preparing the binary classification model
model = ClassificationModel('roberta', 'roberta-base', use_cuda=False, args={'output_dir': 'binary_outputs2/', 'num_train_epochs': 3})

In [75]:
#Training the binary classification model
model.train_model(train_df)

Features loaded from cache at cache_dir/cached_train_roberta_128_2_4000


Running loss: 0.216716

Running loss: 0.045333

Running loss: 0.007530
Training of roberta model complete. Saved to binary_outputs2/.


In [76]:
#Evaluating the binary classification model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Features loaded from cache at cache_dir/cached_dev_roberta_128_2_1000


In [77]:
model.predict(['High income flux expected.'])

Converting to features started.


(array([1]), array([[-1.0868963,  1.1006033]], dtype=float32))

In [78]:
#Viewing the metrics
result

{'mcc': 0.49164735977677965,
 'tp': 108,
 'tn': 739,
 'fp': 74,
 'fn': 79,
 'eval_loss': 0.6333087988737971}

In [23]:
import numpy as np
from sklearn.metrics import label_ranking_average_precision_score

#Comparing the zero rule algorithm accuracy with model accuracy - model accuracy achieves 84.7%
#Which is significantly higher than the zero rule algorithm

def zero_rule_algorithm(train_df, eval_df):
    outputs = [t for t in train_df['labels']]
    most_common = max(set(outputs), key = outputs.count)
    return [most_common for i in range(len(eval_df))]
    
predictions = zero_rule_algorithm(train_df, eval_df)
values = [i for i in eval_df['labels']]
label_ranking_average_precision_score(values, predictions)

0.6890925861870033